<a href="https://colab.research.google.com/github/Pratick9/Meesho_Payment_Reconcillation/blob/main/Meesho_Payment_Reconcillation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from tabulate import tabulate
data = pd.read_csv("/content/sample_data/Test.csv")
data

In [ ]:
print(data.info()) 

In [ ]:
data.Status = data.Status.fillna('Doubt')

In [ ]:
for i in range(len (data.Status)):
  if((data.Status[i]=="Shipped" or data.Status[i]=="Doubt" )and data.Price[i]>0):
    data.Status[i]="Delivered"
  if(data.Status[i]=="Exchange" and data.Price[i]<=0):
    data.Status[i]="Doubt"

In [ ]:
def calculate(df,uni_el,uni_st):
  sum=[]
  x=[]
  for i in uni_el:
    ctr=0
    for colname,colvalue in df.transpose().iterrows():
      if(colname=="Delivered" or colname=="Shipped" or colname=="Exchange" or colname=="Qty"):
        ctr+=df.at[i,colname]
    sum.append(ctr)
    x.append("=")
  return(sum,x)
      

In [ ]:
uni_el=data.SKU.unique()
d=[]
for i in uni_el:
  ctr=0
  df_qty=data[(data.SKU==i)&(data.Qty>1)&(data.Status==('Delivered'or'Shipped' or 'Exchange'))]
  for j in df_qty.Qty:
    ctr+=j-1
  d.append(ctr)
print(d)

In [ ]:
def switch(ch):
  if ch==1:
    uni_el=data.SKU.unique()
    print(*uni_el,sep=" , ")
    inp=input("Enter the Sku,from above, you want to check")
    status=input("Enter the Status you want check")
    dfn=data[(data.SKU==inp)&(data.Status==status)]
    rs=0
    for i in dfn.Price:
      rs=rs+i
    print("Data to searched for : \t",inp,"\t",status)
    print("Total Price : Rs ",rs)
    print("Total count : ",len(dfn))
  else:
    all_poss=[]
    uni_el=data.SKU.unique()
    uni_st=data.Status.unique()
    for i in range (len(uni_el)):
      for j in range (len(uni_st)):
        all_poss.append(data[(data.SKU==uni_el[i])&(data.Status==uni_st[j])])
    col=[]
    ctr=0
    for i in range (len(uni_el)):
      l=[]
      for j in range(len(uni_st)*i,len(uni_st)*(i+1)):  
        l.append(len(all_poss[j]))
      col.append(l)
    df = pd.DataFrame(col, columns =uni_st , dtype = int) 
    df=df.set_index(uni_el)
    df['Qty']=d
    sum,x=calculate(df,uni_el,uni_st)
    df['']=x
    df['Sum']=sum
    print(df)



In [ ]:
ch=int(input("Enter Choice 0.For All 1.For Specific "))
switch(ch)

In [ ]:
rs=0
for i in data.Price:
  rs=rs+i
print("Total Sale of the month : Rs ",rs)